In [1]:
# Alle benötigten Bibliotheken importieren
import pandas as pd
from understatapi import UnderstatClient 

client = UnderstatClient()

In [2]:
# Team und Saison als team_name und saison definieren
team_name = "Freiburg"
saison = "2024"

# Hier wird mittels des Clients die Spielerdaten des vorher definierten Teams (Freiburg) für die definierte Saison abgerufen
team_data = client.team(team=team_name).get_player_data(season=saison)

In [3]:
# In "player_ids" speichere ich die jeweiligen IDs aller SC Spieler in "team_data"
player_ids = [player["id"] for player in team_data]

# Ein leeres Dictionary erstellen welches gleich für die "for"-Schleife benötigt wird.
all_players_shot_data = {}

# Für jeden Spieler in "player_ids" werden jetzt die Schussdaten abgefragt, in "shot_data" gespeichert und dann in das leere Dictionary eingefügt
for Spieler in player_ids:
    shot_data = client.player(player=str(Spieler)).get_shot_data()
    all_players_shot_data[Spieler] = shot_data


In [4]:
# Hier erstelle ich nun eine leere Liste (ähnliches Prinzip wie vorher mit dem Dictionary)
all_shots_list = []

# Hier Iteriere ich mittels einer "for"-Schleife durch alle Spieler und deren Schussdaten
for player_id, shots in all_players_shot_data.items():
    for shot in shots:
        
        # Füge jedem Schuss die zugehörige Spieler-ID hinzu
        shot["player_id"] = player_id 
        
        # Füge den Schuss der leeren Liste hinzu
        all_shots_list.append(shot)

# Die Liste wird nun zu einem DataFrame umgewandelt      
shot_data_df_all = pd.DataFrame(all_shots_list)

In [5]:
# Da aktuell jeder Datentyp "objects" sind, müssen wir diese so verändern, dass diese in Zahlen (float und integers) umgewandelt werden

# Hier werden alle Spalten mit "number" oder "object" ausgewählt und versucht in Zahlen umzuwandeln, durch notna().all() werden alle Spalten wo keine NaN Werte entstehen geprüft
num_cols = shot_data_df_all.select_dtypes(include=["number", "object"]).columns[shot_data_df_all.apply(lambda x: pd.to_numeric(x, errors="coerce").notna().all())]

# Alle Spalten die Problemlos umwandelbar waren werden nun in "integers" umgewandelt
shot_data_df_all[num_cols] = shot_data_df_all[num_cols].apply(pd.to_numeric, downcast="integer")

# Zur Überprüfung habe ich hier die .info() funktion genutzt
shot_data_df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3365 entries, 0 to 3364
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               3365 non-null   int32  
 1   minute           3365 non-null   int8   
 2   result           3365 non-null   object 
 3   X                3365 non-null   float64
 4   Y                3365 non-null   float64
 5   xG               3365 non-null   float64
 6   player           3365 non-null   object 
 7   h_a              3365 non-null   object 
 8   player_id        3365 non-null   int16  
 9   situation        3365 non-null   object 
 10  season           3365 non-null   int16  
 11  shotType         3365 non-null   object 
 12  match_id         3365 non-null   int16  
 13  h_team           3365 non-null   object 
 14  a_team           3365 non-null   object 
 15  h_goals          3365 non-null   int8   
 16  a_goals          3365 non-null   int8   
 17  date          

In [6]:
# Weil ich später die X und Y Koordinaten im Bereich von 0-100 brauche und nicht von 0-1 multipliziere ich die Spalten mit 100
shot_data_df_all[["X", "Y"]] = shot_data_df_all[["X", "Y"]] * 100

# Wieder Prüfen ob es geklappt hat
shot_data_df_all.head()

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,season,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
0,127656,37,BlockedShot,86.699997,58.400002,0.099822,Vincenzo Grifo,a,5266,OpenPlay,2016,LeftFoot,1334,Hertha Berlin,Freiburg,2,1,2016-08-28 17:30:00,Maximilian Philipp,Pass
1,127657,53,BlockedShot,85.199997,76.099998,0.021942,Vincenzo Grifo,a,5266,OpenPlay,2016,RightFoot,1334,Hertha Berlin,Freiburg,2,1,2016-08-28 17:30:00,None,None
2,127662,68,MissedShots,73.699997,62.000000,0.018724,Vincenzo Grifo,a,5266,OpenPlay,2016,RightFoot,1334,Hertha Berlin,Freiburg,2,1,2016-08-28 17:30:00,Nicolas Höfler,Pass
3,128073,4,SavedShot,79.699997,29.200001,0.024902,Vincenzo Grifo,h,5266,OpenPlay,2016,LeftFoot,1340,Freiburg,Borussia M.Gladbach,3,1,2016-09-10 17:30:00,Pascal Stenzel,Pass
4,128079,36,BlockedShot,72.599998,64.199997,0.044127,Vincenzo Grifo,h,5266,DirectFreekick,2016,RightFoot,1340,Freiburg,Borussia M.Gladbach,3,1,2016-09-10 17:30:00,None,Standard


In [7]:
# Nun filter ich die Daten nach der benötigten Saison
shot_data_df_all = shot_data_df_all[shot_data_df_all["season"]==2024]

In [8]:
# Mit Pandas Speichere ich die Daten nun als CSV-Datei
shot_data_df_all.to_csv("Schussdaten_SC_Spieler_2024_25.csv", index=False)